In [ ]:
import datetime
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import expand_dims
import os

import random as python_random

from sklearn.model_selection import train_test_split
# from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array 

import zipfile

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(123)

%load_ext tensorboard

# Het importeren van de data



In [ ]:
# Train -> original given data
# data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Train'

# Train_augumented_dataset
# data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Train_augumented_dataset'

# Train_cleaned_dataset
# data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/imageClassifier/data/Train_cleaned_dataset'

# Train_cleaned_and_augumented
data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Train_cleaned_and_augumented'

batch_size = 32
img_height = 180
img_width = 180

# Split dataset

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2, # voor de trainset neemt tf 80% mee.
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,  # voor de validation set neemt tf 20% mee.
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size) 

class_names = train_ds.class_names
print(class_names)

In [ ]:
#PLOT IMAGES FROM TRAINSET
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
    
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


# Building the model

In [ ]:
# Model 1

# model = Sequential()
# num_classes = len(class_names)

# model.add(layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))

# model.add(layers.Conv2D(32, 3, activation='relu'))
# model.add(layers.MaxPooling2D())

# model.add(layers.Conv2D(32, 3, activation='relu'))
# model.add(layers.MaxPooling2D())

# model.add(layers.Conv2D(32, 3, activation='relu'))
# model.add(layers.MaxPooling2D())

# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(num_classes))

# model.summary()

In [ ]:
# #model 2 geinspireerd van nick nochnack

# model = Sequential()
# num_classes = len(class_names)

# model.add(layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(layers.Dense(num_classes))

# model.summary()

In [ ]:
# Model 3 from paper

model = Sequential()
num_classes = len(class_names)

model.add(layers.Rescaling(1./255,input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(2592))
model.add(layers.Dense(256))

model.add(layers.Dense(num_classes))

model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.summary()

# TRAINING THE MODEL

In [ ]:
logdir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/logs/logTEST' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)



hist = model.fit(train_ds, epochs = 1, validation_data=val_ds, callbacks=[tensorboard_callback])


In [ ]:
# PLOT OF LOSS
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# PLOT OF ACCUARCY
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Het opslaan van een model


In [ ]:
model.save(os.path.join(f'models/[new]px.h5'))

In [ ]:
# new_model = load_model('[NAME]_new.h5')

In [ ]:
# new_model.predict(np.expand_dims(resize/255, 0))

# Prediction

In [ ]:
openImage1 = load_img('/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Test/Rot_Apple/2an7852.jpg',target_size=(img_height, img_width))
                 
def makePrediction(openImage): 
    
    img = np.asarray(openImage).astype('uint8')/255 # normaliseren van de foto.
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    # model = tf.keras.models.load_model(cnn_trained) # inladen trainingsmodel via keras
    
    prediction = model.predict(img_array)
    score = tf.nn.softmax(prediction[0])
    print('This image most likely belongs to {} with a {:.2f} percent confidence.'
         .format(class_names[np.argmax(score)], 100 * np.max(score)),'\n'
         )
    plt.imshow(img)
    
makePrediction(openImage1)

# Evaluate

In [ ]:
test_data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Test'

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
scores = model.evaluate(test_ds)